# SELECT Tutorial


MLDB comes with a powerful [SQL-like Select Query](/doc/#builtin/sql/Sql.md.html) implementation accessible via its [REST API](/doc/#builtin/sql/QueryAPI.md.html). This tutorial will show a few different ways to query data.

The notebook cells below use `pymldb`; you can check out the [Using `pymldb` Tutorial](/doc/nblink.html#_tutorials/Using pymldb Tutorial) for more details.

In [1]:
from pymldb import Connection
mldb = Connection()

# `SELECT`

All queries start with the keyword `SELECT`. Here is the simplest possible query: we ask for 1 and we get a very short result set consisting of one row with one column named 1 and the single cell in it also contains 1.

In [203]:
mldb.query("""

select 1

""")

,1
_rowName,
,1


Of course we can ask for more: the query below does a little math and shows how you can rename your columns with the `as` keyword. Note that single-quotes (`'`) are used to denote strings and double-quotes (`"`) denote column names, both of which can contain any Unicode character.

In [2]:
mldb.query("""

select 1+1, 3+4 as var, 'UTF8 striñg' as "hello, François"

""")

,1+1,var,"hello, François"
_rowName,,,
,2,7,UTF8 striñg


# `FROM` and `LIMIT`

Queries are mostly useful when run on actual datasets, so let's import the passenger manifest from the Titanic.

In [4]:
print mldb.put('/v1/datasets/titanic', { 
    "type": "text.csv.tabular",
    "params": { 
        "dataFileUrl": "https://raw.githubusercontent.com/datacratic/mldb-pytanic-plugin/master/titanic_train.csv" 
    } 
})

<Response [201]>


Now let's query all columns with the star (`*`) operator `FROM` our `titanic` dataset, using the `LIMIT` keyword to avoid getting too much output.

In [6]:
mldb.query("""

select *
from titanic
limit 10

""")

,PassengerId,label,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
_rowName,,,,,,,,,,,,
97,96,0,3,ShorneyMr.CharlesJoseph,male,NaN,0,0,374910,8.0500,None,S
273,272,1,3,TornquistMr.WilliamHenry,male,25,0,0,LINE,0.0000,None,S
524,523,0,3,LahoudMr.Sarkis,male,NaN,0,0,2624,7.2250,None,C
278,277,0,3,LindblomMiss.AugustaCharlotta,female,45,0,0,347073,7.7500,None,S
211,210,1,1,BlankMr.Henry,male,40,0,0,112277,31.0000,A31,C
210,209,1,3,"CarrMiss.Helen""Ellen""",female,16,0,0,367231,7.7500,None,Q
11,10,1,2,NasserMrs.Nicholas(AdeleAchem),female,14,1,0,237736,30.0708,None,C
281,280,1,3,AbbottMrs.Stanton(RosaHunt),female,35,1,1,C.A.2673,20.2500,None,S
698,697,0,3,KellyMr.James,male,44,0,0,363592,8.0500,None,S


We can also ask for just certain columns by name.

In [7]:
mldb.query("""

select Name, Pclass
from titanic
limit 10
""")

,Name,Pclass
_rowName,,
97,ShorneyMr.CharlesJoseph,3
273,TornquistMr.WilliamHenry,3
524,LahoudMr.Sarkis,3
278,LindblomMiss.AugustaCharlotta,3
211,BlankMr.Henry,1
210,"CarrMiss.Helen""Ellen""",3
11,NasserMrs.Nicholas(AdeleAchem),2
281,AbbottMrs.Stanton(RosaHunt),3
698,KellyMr.James,3


# `ORDER BY`

When we've used the `LIMIT` keyword above, it looked like we were just getting a random set of rows. Using the `ORDER BY` keyword we can ask for the 'top 10' according to some criterion.

In [13]:
mldb.query("""

select Name, Age
from titanic
order by Age desc 
limit 10

""")

,Name,Age
_rowName,,
632,BarkworthMr.AlgernonHenryWilson,80.0
853,SvenssonMr.Johan,74.0
98,GoldschmidtMr.GeorgeB,71.0
495,ArtagaveytiaMr.Ramon,71.0
118,ConnorsMr.Patrick,70.5
674,MitchellMr.HenryMichael,70.0
747,CrosbyCapt.EdwardGifford,70.0
35,WheadonMr.EdwardH,66.0
282,DuaneMr.Frank,65.0


# `WHERE`

Beyond limiting the number of records, sometimes we want to look at records which match certain criteria, which we can do with the `WHERE` keyword.

In [14]:
mldb.query("""

select *
from titanic
where Pclass in (1,3) and Sex='male' and (SibSp=1 or Parch=1) and label=1 and Age is null

""")

,PassengerId,label,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
_rowName,,,,,,,,,,,,
711,710,1,3,"MoubarekMaster.HalimGonios(""WilliamGeorge"")",male,None,1,1,2661,15.2458,None,C
67,66,1,3,MoubarekMaster.Gerios,male,None,1,1,2661,15.2458,None,C


# Functions and Aggregate Functions

MLDB comes with a number of builtin functions to operate on your data. Here's an example where we convert a string to uppercase and lowercase.

In [43]:
mldb.query("""

select Name, upper(Name), lower(Name)
from titanic
order by Age desc limit 10

""")

,Name,upper(Name),lower(Name)
_rowName,,,
632,BarkworthMr.AlgernonHenryWilson,BARKWORTHMR.ALGERNONHENRYWILSON,barkworthmr.algernonhenrywilson
853,SvenssonMr.Johan,SVENSSONMR.JOHAN,svenssonmr.johan
98,GoldschmidtMr.GeorgeB,GOLDSCHMIDTMR.GEORGEB,goldschmidtmr.georgeb
495,ArtagaveytiaMr.Ramon,ARTAGAVEYTIAMR.RAMON,artagaveytiamr.ramon
118,ConnorsMr.Patrick,CONNORSMR.PATRICK,connorsmr.patrick
674,MitchellMr.HenryMichael,MITCHELLMR.HENRYMICHAEL,mitchellmr.henrymichael
747,CrosbyCapt.EdwardGifford,CROSBYCAPT.EDWARDGIFFORD,crosbycapt.edwardgifford
35,WheadonMr.EdwardH,WHEADONMR.EDWARDH,wheadonmr.edwardh
282,DuaneMr.Frank,DUANEMR.FRANK,duanemr.frank


The `avg()` function below is a special kind of function: it's an aggregate function, so it operates on multiple rows and gives you a single output.

In [18]:
mldb.query("""

select avg(Age)
from titanic

""")

,avg(Age)
_rowName,
[],29.699118


# GROUP BY & HAVING

You can get aggregate functions to return multiple rows by grouping the input according to some criteria with the `GROUP BY` keyword

In [19]:
mldb.query("""

select Pclass, avg(Age)
from titanic
group by Pclass

""")

,Pclass,avg(Age)
_rowName,,
[1],1,38.233441
[2],2,29.877630
[3],3,25.140620


The `count()` aggregate function is one of the most useful functions and it's a little special: it counts the number of non-null elements in its input. It can also take a star (`*`) to count all rows in its input.

In [20]:
mldb.query("""

select Pclass, count(Age), count(*)
from titanic
group by Pclass

""")

,Pclass,count(Age),count(*)
_rowName,,,
[1],1,186,216
[2],2,173,184
[3],3,355,491


The `HAVING` clause is a little bit like a `WHERE` clause which is applied after `GROUP BY`.

In [21]:
mldb.query("""

select Pclass, avg(Age)
from titanic
group by Pclass
having avg(Age) > 30

""")

,Pclass,avg(Age)
_rowName,,
[1],1,38.233441


# Advanced `FROM` with subqueries

You can get a similar effect to the `HAVING` clause (for a performance hit) by using a second query within your `FROM` clause:

In [22]:
mldb.query("""

select *
from (
    select Pclass, avg(Age) as mean_age
    from titanic
    group by Pclass
)
where mean_age > 30

""")

,Pclass,mean_age
_rowName,,
[1],1,38.233441


# `INTO`: supported via `transform` Procedures 

Other databases support an `INTO` keyword to create new datasets from the output of queries. MLDB does not support this, so as to make `SELECT` queries idempotent, but you can accomplish the same task with a `transform` procedure.

In [23]:
not_supported = """
    select Pclass, avg(Age) as mean_age
    into class_stats
    from titanic
    group by Pclass
"""

supported = mldb.post('/v1/procedures', { 
    "type": "transform",
    "params": { 
        "inputData": """
                select Pclass, avg(Age) as mean_age
                from titanic
                group by Pclass
        """,
        "outputDataset": "class_stats",
        "runOnCreation": True
    } 
})

print supported

<Response [201]>


We can now query our new table!

In [24]:
mldb.query("""

select *
from class_stats

""")

,Pclass,mean_age
_rowName,,
[1],1,38.233441
[3],3,25.140620
[2],2,29.877630


# Advanced `FROM` with `JOIN`

You can run queries across multiple tables with the `JOIN` keyword.

In [25]:
mldb.query("""

select titanic.Name, titanic.Pclass, class_stats.*
from 
    titanic JOIN class_stats 
        ON titanic.Pclass = class_stats.Pclass    
    
order by Age desc limit 10

""")

,titanic.Name,titanic.Pclass,class_stats.Pclass,class_stats.mean_age
_rowName,,,,
[835]-[[3]],AugustssonMr.Albert,3,3,25.14062
[60]-[[2]],WestMiss.ConstanceMirium,2,2,29.87763
[78]-[[3]],StaneffMr.Ivan,3,3,25.14062
[356]-[[3]],YousifMr.Wazli,3,3,25.14062
[134]-[[3]],RobinsMrs.AlexanderA(GraceCharityLaury),3,3,25.14062
[740]-[[3]],IvanoffMr.Kanio,3,3,25.14062
[652]-[[3]],MitkoffMr.Mito,3,3,25.14062
[10]-[[3]],JohnsonMrs.OscarW(ElisabethVilhelminaBerg),3,3,25.14062
[249]-[[2]],HamalainenMrs.William(Anna),2,2,29.87763


----------


# MLDB extensions to conventional SQL

Selecting columns based on a prefix.

In [45]:
mldb.query("""

select P*
from titanic
limit 1

""")

,PassengerId,Pclass,Parch
_rowName,,,
97,96,3,0


Renaming columns based on a prefix pattern

In [27]:
mldb.query("""

select P* as x*
from titanic
limit 1

""")

,xassengerId,xclass,xarch
_rowName,,,
97,96,3,0


Excluding columns from a select.

In [28]:
mldb.query("""

select * excluding(P*)
from titanic
limit 1

""")

,label,Name,Sex,Age,SibSp,Ticket,Fare,Cabin,Embarked
_rowName,,,,,,,,,
97,0,ShorneyMr.CharlesJoseph,male,None,0,374910,8.05,None,S


NOTE: Selecting a column which is not in the dataset will not cause an error, but it will return `NULL`.

In [29]:
mldb.query("""

select nothing
from titanic
limit 1

""")

,nothing
_rowName,
97,None


MLDB supports JSON-like objects in queries.

In [30]:
mldb.query("""

select {a: 1, b:2, c: 'hello'}

""")

,"{a: 1, b:2, c: 'hello'}.a","{a: 1, b:2, c: 'hello'}.b","{a: 1, b:2, c: 'hello'}.c"
_rowName,,,
,1,2,hello


In [31]:
mldb.query("""

select {a: 1, b:2, c: 'hello'} as obj

""")

,obj.a,obj.b,obj.c
_rowName,,,
,1,2,hello


In [32]:
mldb.query("""

select {a: 1, b:{x:2}, c: 'hello'} as *

""")

,a,b.x,c
_rowName,,,
,1,2,hello


This is especially useful for tokenizing text into bags of words, or importing semi-structured JSON data.

In [50]:
mldb.query("""

select tokenize('Hello world, Hello!', {splitchars: ' ,!'}) as *

""")

,world,Hello
_rowName,,
,1,2


In [51]:
mldb.query("""

select parse_json('{"hello":"world","list":[1,2,3,4]}') as *

""")

,hello,list.0,list.1,list.2,list.3
_rowName,,,,,
,world,1,2,3,4


MLDB's object notation also allows you to run aggregates on multiple columns at once.

In [52]:
mldb.query("""

select Pclass, count(*), count({*})
from titanic
group by Pclass

""")

,Pclass,count(*),count({*}).Age,count({*}).Cabin,count({*}).Embarked,count({*}).Fare,count({*}).Name,count({*}).Parch,count({*}).PassengerId,count({*}).Pclass,count({*}).Sex,count({*}).SibSp,count({*}).Ticket,count({*}).label
_rowName,,,,,,,,,,,,,,
[1],1,216,186,176,214,216,216,216,216,216,216,216,216,216
[2],2,184,173,16,184,184,184,184,184,184,184,184,184,184
[3],3,491,355,12,491,491,491,491,491,491,491,491,491,491


MLDB's flexible output model enables powerful functions like `pivot()` to exist.

In [53]:
mldb.query("""

select Pclass, pivot(Sex, "count(*)") as *
from (
    select Pclass, Sex, count(*)
    from titanic
    group by Pclass, Sex
)
group by Pclass

""")

,Pclass,female,male
_rowName,,,
[1],1,94,122
[2],2,76,108
[3],3,144,347


MLDB supports multi-dimensional arrays called embeddings, also known as tensors.

In [55]:
mldb.query("""

select [1,2,3] as embedding

""")

,embedding.0,embedding.1,embedding.2
_rowName,,,
,1,2,3


In [56]:
mldb.query("""

select normalize([1,2,3], 1) as n,  [1,2,3] / norm([1,2,3] ,1) as d

""")

,n.0,n.1,n.2,d.0,d.1,d.2
_rowName,,,,,,
,0.166667,0.333333,0.5,0.166667,0.333333,0.5


MLDB datasets have named rows as well as columns, and the `NAMED` keyword allows you to control the names of your output rows.

In [39]:
mldb.query("""

select *
named Name + ' aged ' + cast(Age as string)
from titanic
order by Age desc limit 10

""")

,PassengerId,label,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
_rowName,,,,,,,,,,,,
ShorneyMr.CharlesJoseph aged,96,0,3,ShorneyMr.CharlesJoseph,male,NaN,0,0,374910,8.0500,None,S
TornquistMr.WilliamHenry aged 25,272,1,3,TornquistMr.WilliamHenry,male,25,0,0,LINE,0.0000,None,S
LahoudMr.Sarkis aged,523,0,3,LahoudMr.Sarkis,male,NaN,0,0,2624,7.2250,None,C
LindblomMiss.AugustaCharlotta aged 45,277,0,3,LindblomMiss.AugustaCharlotta,female,45,0,0,347073,7.7500,None,S
BlankMr.Henry aged 40,210,1,1,BlankMr.Henry,male,40,0,0,112277,31.0000,A31,C
"CarrMiss.Helen""Ellen"" aged 16",209,1,3,"CarrMiss.Helen""Ellen""",female,16,0,0,367231,7.7500,None,Q
NasserMrs.Nicholas(AdeleAchem) aged 14,10,1,2,NasserMrs.Nicholas(AdeleAchem),female,14,1,0,237736,30.0708,None,C
AbbottMrs.Stanton(RosaHunt) aged 35,280,1,3,AbbottMrs.Stanton(RosaHunt),female,35,1,1,C.A.2673,20.2500,None,S
KellyMr.James aged 44,697,0,3,KellyMr.James,male,44,0,0,363592,8.0500,None,S


Having named rows as well as columns allows us to easily operate on the transpose of a dataset

In [40]:
mldb.query("""

select * from transpose(
    (
        select *
        named Name + ' aged ' + cast(Age as string)
        from titanic
        order by Age desc limit 10
    ) as original
)

""")

,ShorneyMr.CharlesJoseph aged,TornquistMr.WilliamHenry aged 25,LahoudMr.Sarkis aged,LindblomMiss.AugustaCharlotta aged 45,BlankMr.Henry aged 40,"CarrMiss.Helen""Ellen"" aged 16",NasserMrs.Nicholas(AdeleAchem) aged 14,AbbottMrs.Stanton(RosaHunt) aged 35,KellyMr.James aged 44,ToomeyMiss.Ellen aged 50
_rowName,,,,,,,,,,
Cabin,None,None,None,None,A31,None,None,None,None,None
Fare,8.05,0,7.225,7.75,31,7.75,30.0708,20.25,8.05,10.5
SibSp,0,0,0,0,0,0,1,1,0,0
Ticket,374910,LINE,2624,347073,112277,367231,237736,C.A.2673,363592,F.C.C.13531
PassengerId,96,272,523,277,210,209,10,280,697,459
label,0,1,0,0,1,1,1,1,0,1
Age,None,25,None,45,40,16,14,35,44,50
Pclass,3,3,3,3,1,3,2,3,3,2
Name,ShorneyMr.CharlesJoseph,TornquistMr.WilliamHenry,LahoudMr.Sarkis,LindblomMiss.AugustaCharlotta,BlankMr.Henry,"CarrMiss.Helen""Ellen""",NasserMrs.Nicholas(AdeleAchem),AbbottMrs.Stanton(RosaHunt),KellyMr.James,ToomeyMiss.Ellen


MLDB supports inline Javascript application via the `jseval()` function.

In [41]:
mldb.query("""

select Name,
    jseval('
        return Name.replace(/([A-Z])/g, function(m, p) { return " "+p; });
    ', 'Name', Name) as processed_name
from titanic
order by Age desc limit 10

""")

,Name,processed_name
_rowName,,
97,ShorneyMr.CharlesJoseph,Shorney Mr. Charles Joseph
273,TornquistMr.WilliamHenry,Tornquist Mr. William Henry
524,LahoudMr.Sarkis,Lahoud Mr. Sarkis
278,LindblomMiss.AugustaCharlotta,Lindblom Miss. Augusta Charlotta
211,BlankMr.Henry,Blank Mr. Henry
210,"CarrMiss.Helen""Ellen""","Carr Miss. Helen"" Ellen"""
11,NasserMrs.Nicholas(AdeleAchem),Nasser Mrs. Nicholas( Adele Achem)
281,AbbottMrs.Stanton(RosaHunt),Abbott Mrs. Stanton( Rosa Hunt)
698,KellyMr.James,Kelly Mr. James


MLDB datasets handle millions of columns, and deal very well with sparse datasets, making them ideal for operating on bags of words.

In [57]:
mldb.query("""

select tokenize(
    jseval('
        return Name.replace(/([A-Z])/g, function(m, p) { return " "+p; });
    ', 'Name', Name),
    {splitchars: ' .()"', quotechar:''}) as *
from titanic
order by Age desc
limit 10

""")

,Henry,Algernon,Mr,Wilson,Barkworth,Johan,Svensson,B,George,Goldschmidt,...,Edward,Capt,Crosby,H,Wheadon,Frank,Duane,Davis,Francis,Millet
_rowName,,,,,,,,,,,,,,,,,,,,,
632,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,NaN,NaN,1,NaN,NaN,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,1,NaN,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
282,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


Putting it all together, here are the top 20 tokens present in the names of Titanic passengers.

In [58]:
mldb.query("""

select * from transpose(
    (
        select sum(
            tokenize(
                jseval('
                    return Name.replace(/([A-Z])/g, function(m, p) { return " "+p; });
                ', 'Name', Name),
                {splitchars: ' .()"', quotechar:''})
            ) as *
        named 'counts'
        from titanic
    )
)
order by counts desc limit 20

""")

,counts
_rowName,
Mr,521
Miss,182
Mrs,128
William,64
John,44
Master,40
Henry,35
George,24
James,24


## Where to next?

Check out the other [Tutorials and Demos](/doc/#builtin/Demos.md.html).